In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt 
import matplotlib.patches as patches

 

In [ ]:
def step_Domain(x0,x1,y0,y1,dx,dy):
    X = np.arange(x0,x1,dx)
    Y = np.arange(y0,y1,dy)
    xx,yy = np.meshgrid(X,Y)
    u = np.zeros((Y.size+2,X.size+2))
    v = np.zeros((Y.size+2,X.size+2))
    p = np.zeros((Y.size+2,X.size+2))
    return xx,yy,u,v,p

def intial(u,v,p,xx,yy,level=25):
    
    plt.figure()
    plt.contourf(xx,yy,u(xx,yy),cmap='jet',levels=level)
    plt.title('Distribution of U ')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.colorbar()

    plt.figure()
    plt.contourf(xx,yy,v(xx,yy),cmap='jet',levels=level)
    plt.title('Distribution of V ')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.colorbar()

    
    plt.figure()
    plt.contourf(xx,yy,p(xx,yy),cmap='jet',levels=level)
    plt.colorbar()
    plt.title('Distribution of P ')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    plt.figure()
    plt.streamplot(xx,yy,u(xx,yy),v(xx,yy),color='black')
    plt.contourf(xx,yy,p(xx,yy),cmap='jet',levels=level)
    plt.colorbar()
    plt.title('Distribution')
    plt.xlabel('X')
    plt.ylabel('Y')
    

In [ ]:
P = lambda x,y : 0*x*y
U = lambda x,y : 3*(x*y)**0
V = lambda x,y : 0*x*y

In [ ]:
dx = 0.05
dy = 0.05

In [ ]:
xx,yy,u,v,p = step_Domain(0,4,0,1,dx,dy)
intial(U,V,P,xx,yy)
P_I = P(xx,yy)
U_I = U(xx,yy)
V_I = V(xx,yy)

In [ ]:
beta = 1
rho = 1
Uw = 1
Lr = 1
mu = 0.000001 

# Re = (rho*Uw*Lr)/(mu)
Re = 10

In [ ]:
def solution_vector(u):
    s = np.ndarray.tolist(np.copy(u))
    for i in range(0,np.shape(u)[0]):
        for j in range(0,np.shape(u)[1]):
            s[i][j]  = [0,0,0]
    return s 

s = solution_vector(u)
f = solution_vector(u)
g = solution_vector(u)

In [ ]:
def mask_fn(x0_s,x1_s,y0_s,y1_s,u,dx,dy):
    mask = np.ones(u.shape)
    L_x = x1_s - x0_s
    L_y = y1_s - y0_s
    n_start_setp_x = round((x0_s-0)/dx)
    n_start_setp_y = round((y0_s-0)/dy)
    n_end_setp_x = round(n_start_setp_x+(L_x)/dx)
    n_end_setp_y = round(n_start_setp_y+(L_y)/dy)
    
    for i in range(n_start_setp_y+1,n_end_setp_y+1):
        for j in range(n_start_setp_x+1,n_end_setp_x+1):
            mask[i,j] = 0
    return mask,n_start_setp_x,n_start_setp_y,n_end_setp_x,n_end_setp_y
    
mask,n_start_setp_x,n_start_setp_y,n_end_setp_x,n_end_setp_y = mask_fn(0,0.75,0,0.35,u,dx,dy)

In [ ]:
u[1:-1,1:-1] = U_I
v[1:-1,1:-1] = V_I
p[1:-1,1:-1] = P_I

In [ ]:
for i in range(np.shape(s)[0]):
    for j in range(np.shape(s)[1]):
        s[i][j][0] = p[i,j]
        s[i][j][1] = u[i,j]
        s[i][j][2] = v[i,j]


In [ ]:
dt = 0.0001
beta=0.001
dt_order = [1/4,1/3,1/2,1]

for iter in range(2000):
    # Up
    for j in range(np.shape(s[0])[0]):
        s[0][j][1] = -s[1][j][1]
        s[0][j][2] = -s[1][j][2]

    # Down
    for j in range(np.shape(s[0])[0]):
        s[-1][j][1] = -s[-2][j][1]
        s[-1][j][2] = -s[-2][j][2]

    # Left
    for i in range(np.shape(s)[0]):
        s[i][0][1] = 1*2 -s[i][1][1]
        s[i][0][2] = -s[i][1][2]



    # Right
    for i in range(np.shape(s)[0]):
        s[i][-1][1] = s[i][-2][1]
        s[i][-1][2] = s[i][-2][2]
        
        
    # Step Up 
    for i in range(n_start_setp_x - 1, n_end_setp_x - 1):
        s[n_end_setp_y - 1][i][1] = -s[n_end_setp_y][i][1]  
        s[n_end_setp_y - 1][i][2] = -s[n_end_setp_y][i][2]
        s[n_end_setp_y - 1][i][0] = s[n_end_setp_y][i][0]

    # Step Right
    for i in range(n_start_setp_y - 1, n_end_setp_y - 1):
        s[i][n_end_setp_x - 1][1] = -s[i][n_end_setp_x][1]  
        s[i][n_end_setp_x - 1][2] = -s[i][n_end_setp_x][2]  
        s[i][n_end_setp_x - 1][0] = s[i][n_end_setp_x][0]

    s_new = s.copy()
    for dt_o in dt_order:
        for i in range(1, np.shape(s)[0] - 1):
            for j in range(1, np.shape(s)[1] - 1):
                if mask[i,j] == 1:
                    # i,j
                    p_t_ij = s_new[i][j][0]
                    u_t_ij = s_new[i][j][1]
                    v_t_ij = s_new[i][j][2]
                    # i+1,j
                    p_t_e = s_new[i][j + 1][0]
                    u_t_e = s_new[i][j + 1][1]
                    v_t_e = s_new[i][j + 1][2]
                    # i-1,j
                    p_t_w = s_new[i][j - 1][0]
                    u_t_w = s_new[i][j - 1][1]
                    v_t_w = s_new[i][j - 1][2]
                    # i,j+1
                    p_t_n = s_new[i + 1][j][0]
                    u_t_n = s_new[i + 1][j][1]
                    v_t_n = s_new[i + 1][j][2]
                    # i,j-1
                    p_t_s = s_new[i - 1][j][0]
                    u_t_s = s_new[i - 1][j][1]
                    v_t_s = s_new[i - 1][j][2]

                    Fe = [[(u_t_e + u_t_ij) / (2 * beta)],
                        [((u_t_e + u_t_ij) / 2) ** 2 + ((p_t_ij + p_t_e) / 2) - ((1 / Re) * ((u_t_e - u_t_ij) / dx))],
                        [((u_t_e + u_t_ij) / 2) * ((v_t_e + v_t_ij) / 2) - (1 / Re) * ((v_t_e - v_t_ij) / dx)]]

                    Fw = [[(u_t_w + u_t_ij) / (2 * beta)],
                        [((u_t_w + u_t_ij) / 2) ** 2 + ((p_t_ij + p_t_w) / 2) - ((1 / Re) * ((u_t_ij - u_t_w) / dx))],
                        [((u_t_w + u_t_ij) / 2) * ((v_t_w + v_t_ij) / 2) - (1 / Re) * ((v_t_ij - v_t_w) / dx)]]

                    Gn = [[(v_t_n + v_t_ij) / (2 * beta)],
                        [((u_t_n + u_t_ij) / 2) * ((v_t_n + v_t_ij) / 2) - (1 / Re) * ((u_t_n - u_t_ij) / dy)],
                        [((v_t_n + v_t_ij) / 2) ** 2 + ((p_t_n + p_t_ij) / 2) - (1 / Re) * ((v_t_n - v_t_ij) / dy)]]

                    Gs = [[(v_t_s + v_t_ij) / (2 * beta)],
                        [((u_t_s + u_t_ij) / 2) * ((v_t_s + v_t_ij) / 2) - (1 / Re) * ((u_t_ij - u_t_s) / dy)],
                        [((v_t_s + v_t_ij) / 2) ** 2 + ((p_t_s + p_t_ij) / 2) - (1 / Re) * ((v_t_ij - v_t_s) / dy)]]

                    D_F = (np.array(Fe) - np.array(Fw)) / dx
                    D_G = (np.array(Gn) - np.array(Gs)) / dy
                    s_new[i][j] = np.ndarray.tolist((dt*dt_o* (-np.array(D_F) - np.array(D_G))).reshape(1, 3) + (np.array(s[i][j])))[0]
                    
                else:
                    s[i][j][0] = 0
                    s[i][j][1] = 0
                    s[i][j][2] = 0
                    
    if iter % 25 == 0:
        print(f'----------Iter {iter}----------')

    s = s_new


In [ ]:
for j in range(np.shape(s)[0]):
    for i in range(np.shape(s)[1]):
        p[j,i] = s[j][i][0] 
        u[j,i] = s[j][i][1]
        v[j,i] = s[j][i][2]

In [ ]:
plt.figure()
plt.streamplot(xx,yy,u[1:-1,1:-1],v[1:-1,1:-1],color='black',)
# plt.contour(np.sqrt(u[1:-1,1:-1]**2 + v[1:-1,1:-1]**2))
plt.contourf(xx,yy,u[1:-1,1:-1],cmap='jet',levels=25)
square = patches.Rectangle((0,0), 0.75, 0.35, linewidth=0.5, edgecolor='black', facecolor='gray')
plt.gca().add_patch(square)
plt.colorbar()
plt.title('Distribution U')
plt.xlabel('X')
plt.ylabel('Y')
plt.figure()
plt.streamplot(xx,yy,u[1:-1,1:-1],v[1:-1,1:-1],color='black',)
# plt.contour(np.sqrt(u[1:-1,1:-1]**2 + v[1:-1,1:-1]**2))
plt.contourf(xx,yy,v[1:-1,1:-1],cmap='jet',levels=25)
square = patches.Rectangle((0,0), 0.75, 0.35, linewidth=0.5, edgecolor='black', facecolor='gray')
plt.gca().add_patch(square)
plt.colorbar()
plt.title('Distribution V')
plt.xlabel('X')
plt.ylabel('Y')
plt.figure()
plt.streamplot(xx,yy,u[1:-1,1:-1],v[1:-1,1:-1],color='black',)
# plt.contour(np.sqrt(u[1:-1,1:-1]**2 + v[1:-1,1:-1]**2))
plt.contourf(xx,yy,p[1:-1,1:-1],cmap='jet',levels=25)
square = patches.Rectangle((0,0), 0.75, 0.35, linewidth=0.5, edgecolor='black', facecolor='gray')
plt.gca().add_patch(square)
plt.colorbar()
plt.title('Distribution P')
plt.xlabel('X')
plt.ylabel('Y')